In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import mysql.connector

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# conector com o banco
#conn = mysql.connector.connect(user='root', password='rede243', host='localhost', database='ppsus')

Extrai a tabela 'coleta' do banco e divide em tres dataframes (respostas, variáveis, variáveis categóricas)

In [36]:
#df_coleta = pd.read_sql('SELECT * FROM ppsus_app_coleta', conn)
df_coleta = pd.read_csv('data.csv')
cols = df_coleta.columns
n_inst = df_coleta.shape[0]

df_answ = df_coleta.loc[:,'subjetiva':'edmonton_q9']
cols_answ = df_answ.columns

df_feat =  df_coleta.loc[:,'meem':'tnf_beta']
cols_feat = df_feat.columns

df_feat_cat = df_coleta.loc[:,'internacao_cat':'hematocrito_cat']
cols_feat_cat = df_feat_cat.columns

#df_feat =  df_coleta[['meem', 'ativ_fis', 'temp_sent', 'gds', 'fes', 'man', 'fraq_musc_media', 'fraq_musc_max', 'lawton', 'katz', 'aavd', 'tug', 'caminhada', 'mos', 'circ_cint', 'circ_quad', 'circ_pant', 'berg', 'relogio', 'quedas']]
#df_feat_cat = df_coleta[['meem_cat', 'gds_cat2', 'katz_cat2', 'aavd_cat', 'tug_cat', 'berg_cat2']]

## I) Validação cruzada com relação à métrica do critério de Fisher

In [22]:
# calcula o score de fisher para o cross validation
def KFisherScore(df, aval='subjetiva'):
    score = np.zeros((1, len(cols_feat)))
    
    for c1 in ['F', 'P']:
        c2 = ['F', 'P', 'N']
        c2.remove(c1)
        
        df1 = df.query('subjetiva == "'+c1+'"').loc[:,cols_feat]
        df2 = df.query('subjetiva == "'+c2[0]+'" or subjetiva == "'+c2[1]+'"').loc[:,cols_feat]
        
        score += ( (np.mean(df1) - np.mean(df2)).pow(2) / (np.var(df1) + np.var(df2)) ).values
        #score += ( (np.mean(df1) - np.mean(df2)).pow(2) / (np.var(df1, ddof=1) + np.var(df2, ddof=1)) ).values
        
    return score/3

In [142]:
def plotFisherScore(aval, scores):
    mean = scores.mean(axis=0)
    std = scores.std(axis=0)
    # ordenando
    mean, std, feats = zip(*sorted(zip(mean, std, cols_feat), reverse=True))

    data = [go.Bar(x=feats,
                   y=mean,
                   error_y=dict(
                       type='data',
                       array=std,
                       visible=True))]

    fig = go.Figure(data=data)
    fig['layout'].update(title='Pontuação das variáveis no Score de Fisher para a avaliação '+aval)

    iplot(fig)

In [23]:
# faz a validação cruzada com k=10
k_size = int(n_inst*0.1)

cross_validation = np.zeros((10, len(cols_feat)))

for k in range(10):
    a = k*k_size
    b = (k+1) * k_size
    
    s1 = KFisherScore(df_coleta.iloc[a : b+1])
    s2 = KFisherScore( pd.concat((df_coleta.iloc[ : a], df_coleta.iloc[b : ])) )
    
    cross_validation[k] = s1 - s2

In [24]:
# salva as 10 execuções em um arquivo csv
df = pd.DataFrame(cross_validation, range(10), df_coleta.loc[:,'meem':'tnf_beta'].columns)
df.to_csv('cross_validation_fisher_score.csv')

In [143]:
plotFisherScore('subjetiva', cross_validation)

## II) Similaridade de respostas obtidas pela Subjetiva e Edmonton

In [3]:
def distance(x, y, avaliacao):
    if avaliacao == 'subjetiva':
        x = x.loc['subjetiva_q1' : 'subjetiva_q6'].drop('subjetiva_q1_kg')
        y = y.loc['subjetiva_q1' : 'subjetiva_q6'].drop('subjetiva_q1_kg')                
    else:
        x = x.loc['edmonton_q1' : 'edmonton_q9']
        y = y.loc['edmonton_q1' : 'edmonton_q9']
    
    return np.nansum(abs(x - y), dtype=np.int)

In [4]:
#n_inst = 10
#n_inst = df_coleta.shape[0]

dist_sub = np.zeros(18)
dist_edm = np.zeros(18)

for i in range(n_inst-1):
    for j in range(i+1, n_inst):
        d_sub = distance(df_answ.iloc[i], df_answ.iloc[j], 'subjetiva')
        d_edm = distance(df_answ.iloc[i], df_answ.iloc[j], 'edmonton')
        
        dist_sub[d_sub] += 1
        dist_edm[d_edm] += 1

import math
combinations = math.factorial(n_inst)/(2*math.factorial(n_inst-2))
dist_sub /= combinations
dist_edm /= combinations

In [5]:
trace1 = go.Bar(
    x=np.arange(12),
    y=dist_sub,
    name='Subjetiva'
)
trace2 = go.Bar(
    x=np.arange(12),
    y=dist_edm,
    name='Edmonton'
)
data = [trace1, trace2]

layout = go.Layout(
    title='Distribuição das distâncias',
    xaxis=dict(
        title='Distância'
    ),
    yaxis=dict(
        title='Proporção de pares de indivíduos'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

## III) Análise dos padrões de resposta obtidas pela Subjetiva e Edmonton

### Subjetiva

In [98]:
def getClass(aval, vet_ans):
    score = np.nansum(array)
    
    if aval == 'subjetiva':
        if vet_ans[4]==1 and vet_ans[5]==1:
            score -= 1
        if score == 0:
            return 'N'
        elif score < 3:
            return 'P'
        else:
            return 'F'
    else:
        # Não apresenta fragilidade
        if score <= 4:
            return 'N'
        # Aparentemente vulnerável
        elif score <= 6:
            return 'V'
        # Fragilidade leve
        elif score <= 8:
            return 'L'
        # Fragilidade moderada
        elif score <= 10:
            return 'M'
        # Fragilidade severa
        else:
            return 'S'

In [47]:
sub_pad = {'Frágil' : [[], []], 'Pré-frágil' : [[], []], 'Não frágil' : [[], []]}
query = 'subjetiva_q1=={} and subjetiva_q2=={} and subjetiva_q3=={} and subjetiva_q4=={} and subjetiva_q5=={} and subjetiva_q6=={}'
                        
# todas as possíveis combinações de pontuação para a Subjetiva
for i0 in range(2):
    for i1 in range(2):
        for i2 in range(2):
            for i3 in range(2):
                for i4 in range(2):
                    for i5 in range(2):
                        
                        qtd = df_answ.fillna(0).query(query.format(i0, i1, i2, i3, i4, i5)).shape[0]
                        array = [i0, i1, i2, i3, i4, i5]
                        classe = getClass('subjetiva', array)
                        
                        if classe == 'N':
                            sub_pad['Não frágil'][0].append(qtd)
                            sub_pad['Não frágil'][1].append(array)
                        elif classe == 'P':
                            sub_pad['Pré-frágil'][0].append(qtd)
                            sub_pad['Pré-frágil'][1].append(array)
                        else:
                            sub_pad['Frágil'][0].append(qtd)
                            sub_pad['Frágil'][1].append(array)

In [149]:
classes = ['Frágil', 'Pré-frágil', 'Não frágil']

data = [go.Bar(
            x=classes,
            y=[sum(sub_pad[k][0])/n_inst for k in classes]
    )]

layout = go.Layout(
    title='Proporção de indivíduos por classe na Subjetiva',
    xaxis=dict(
        title='Classe'
    ),
    yaxis=dict(
        title='Proporção de indivíduos'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [152]:
traces = []
for k in classes:

    l1, l2 = zip(*sorted(zip(sub_pad[k][0], map(str, sub_pad[k][1])), reverse=True))

    traces.append(go.Bar(
        x = l2,
        y = [y for y in l1]
    ))

fig = tools.make_subplots(rows=3, cols=1, subplot_titles=classes, print_grid=False)

for i, t in enumerate(traces):
    fig.append_trace(t, i+1, 1)

fig['layout'].update(height=900, 
                     title='Quantidade de indivíduos por padrão de pontuação por classe na Subjetiva', 
                     showlegend=False)

iplot(fig)

### Edmonton

In [159]:
edm_pad = {'Severa' : [[], []], 'Moderada' : [[], []], 'Leve' : [[], []], 'Vulnerável' : [[], []], 'Não frágil' : [[], []]}
query = 'edmonton_q1=={} and edmonton_q2_b=={} and edmonton_q2_a=={} and edmonton_q3=={} and edmonton_q4=={} and edmonton_q5_a=={} and edmonton_q5_b=={} and edmonton_q6=={} and edmonton_q7=={} and edmonton_q8=={} and edmonton_q9=={}'               

# todas as possíveis combinações de pontuação para a Edmonton
for i0 in range(3):#3
    for i1 in range(3):#3
        for i2 in range(3):#3
            for i3 in range(3):#3
                for i4 in range(3):#3
                    for i5 in range(2):
                        for i6 in range(2):
                            for i7 in range(2):
                                for i8 in range(2):
                                    for i9 in range(2):
                                        for i10 in range(3):#3
                        
                                            qtd = df_answ.fillna(0).query(query.format(i0, i1, i2, i3, i4, i5, i6, i7, i8, i9, i10)).shape[0]
                                            array = [i0, i1, i2, i3, i4, i5, i6, i7, i8, i9, i10]
                                            classe = getClass('edmonton', array)
                                    
                                            if qtd > 0:                                                
                                                # Não apresenta fragilidade
                                                if classe == 'N':
                                                    edm_pad['Não frágil'][0].append(qtd)
                                                    edm_pad['Não frágil'][1].append(array)

                                                # Aparentemente vulnerável
                                                elif classe == 'V':
                                                    edm_pad['Vulnerável'][0].append(qtd)
                                                    edm_pad['Vulnerável'][1].append(array)

                                                # Fragilidade leve
                                                elif classe == 'L':
                                                    edm_pad['Leve'][0].append(qtd)
                                                    edm_pad['Leve'][1].append(array)

                                                # Fragilidade moderada
                                                elif classe == 'M':
                                                    edm_pad['Moderada'][0].append(qtd)
                                                    edm_pad['Moderada'][1].append(array)

                                                # Fragilidade severa
                                                else:
                                                    edm_pad['Severa'][0].append(qtd)
                                                    edm_pad['Severa'][1].append(array)

In [160]:
classes = ['Severa', 'Moderada', 'Leve', 'Vulnerável', 'Não frágil']

data = [go.Bar(
            x=classes,
            y=[sum(edm_pad[k][0])/n_inst for k in classes]
    )]

layout = go.Layout(
    title='Proporção de indivíduos por classe na Edmonton',
    xaxis=dict(
        title='Classe'
    ),
    yaxis=dict(
        title='Proporção de indivíduos'
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [162]:
traces = []
for k in classes:
    # se tiver instâncias com a classe k
    if len(edm_pad[k][0]):
        l1, l2 = zip(*sorted(zip(edm_pad[k][0], map(str, edm_pad[k][1])), reverse=True))

        traces.append(go.Bar(
            x = l2,
            y = [y for y in l1],
            name=k
        ))
    else:
        traces.append(go.Bar(
            x = [''],
            y = [0]
        ))

fig = tools.make_subplots(rows=5, cols=1, subplot_titles=classes, print_grid=False)

for i, t in enumerate(traces):
    fig.append_trace(t, i+1, 1)

fig['layout'].update(height=2000, 
                     title='Quantidade de indivíduos por padrão de pontuação por classe na Edmonton', 
                     showlegend=False)

iplot(fig)

## I - b) Avaliação do critério de Fisher

### Subjetiva

In [40]:
def getIdsByMaxDistance(aval, vet_answ, df):
    
    cols_aval = {'subjetiva' : ['subjetiva_q'+str(i+1) for i in range(6)],
                 'edmonton'  : ['edmonton_q'+str(i+1) for i in range(9) if i not in {1, 4}] +
                               ['edmonton_q2_a', 'edmonton_q2_b', 'edmonton_q5_a', 'edmonton_q5_b']}
    cols = cols_aval[aval]
    if aval == 'subjetiva':
        dist_max = 2
    else:
        dist_max = 4
    
    # trocando nulos por 0
    not_nan = df[cols].fillna(0)
    
    ret = []
    for index, row in not_nan.iterrows():        
        dist = 0
        for a, b in zip(vet_answ, row):
            dist += abs(a - b)
        if dist <= dist_max:
            ret.append(index)
    return(ret)

In [135]:
# vetor que armazena todas as execuções do score de fisher
scores = np.zeros((2**6, len(cols_feat)))
i = 0

# todas as possíveis combinações de pontuação para a Subjetiva
for i0 in range(2):
    for i1 in range(2):
        for i2 in range(2):
            for i3 in range(2):
                for i4 in range(2):
                    for i5 in range(2):
                        array = [i0, i1, i2, i3, i4, i5]
                        classe = getClass('subjetiva', array)
                        exemplos = getIdsByMaxDistance('subjetiva', array, df_coleta)
                        
                        # nunca acontece =/
                        if len(exemplos) < 0.1*df_coleta.shape[0]:
                            exemplos = df_coleta.query('subjetiva == '+classe).index
                            
                        
                        df_prox = df_feat.loc[exemplos]                            
                        df_dist = df_feat.loc[ ~df_feat.index.isin(exemplos) ]
                        
                        scores[i] = (np.mean(df_prox) - np.mean(df_dist)).pow(2) / (np.var(df_prox) + np.var(df_dist))
                        i += 1

In [138]:
plotFisherScore('subjetiva', scores)

### Edmonton

In [111]:
# convertendo a classificação de F,P,N para L,M,N,S,V
for i, row in df_coleta.iterrows():
    array = row.loc['edmonton_q1':'edmonton_q9'].tolist()    
    classe = getClass('edmonton', array)
    df_coleta.loc[i, 'edmonton'] = classe
    
df_answ['edmonton'] = df_coleta['edmonton']

In [140]:
# vetor que armazena todas as execuções do score de fisher
scores = np.zeros((3**6 * 2**5, len(cols_feat)))
i = 0

# todas as possíveis combinações de pontuação para a Edmonton
for i0 in range(3):#3
    for i1 in range(3):#3
        for i2 in range(3):#3
            for i3 in range(3):#3
                for i4 in range(3):#3
                    for i5 in range(2):
                        for i6 in range(2):
                            for i7 in range(2):
                                for i8 in range(2):
                                    for i9 in range(2):
                                        for i10 in range(3):#3                                                
                                            array = [i0, i1, i2, i3, i4, i5, i6, i7, i8, i9, i10]
                                            classe = getClass('edmonton', array)
                                            exemplos = getIdsByMaxDistance('edmonton', array, df_coleta)

                                            if len(exemplos) < 0.1*df_coleta.shape[0]:
                                                # Severa não tem 10% da base, por isso juntamos com Moderada
                                                if classe=='S':
                                                    exemplos = df_coleta.query('edmonton=="S" or edmonton=="M"').index
                                                else:
                                                    exemplos = df_coleta.query('edmonton=="'+classe+'"').index
                                                    
                                            df_prox = df_feat.loc[exemplos]
                                            df_dist = df_feat.loc[ ~df_feat.index.isin(exemplos) ]
                        
                                            scores[i] = (np.mean(df_prox) - np.mean(df_dist)).pow(2) / (np.var(df_prox) + np.var(df_dist))
                                            i += 1

In [141]:
plotFisherScore('edmonton', scores)